## Creating Medallion Architecture

In [0]:
%sql
-- CREATE SCHEMA IF NOT EXISTS enterprise_modernization.bronze;
-- CREATE SCHEMA IF NOT EXISTS enterprise_modernization.silver;
-- CREATE SCHEMA IF NOT EXISTS enterprise_modernization.gold;


## Bronze to Silver transformation

### SAP

In [0]:
sap_bronze = spark.table("enterprise_modernization.bronze.bronze_sap")
sap_silver = (
    sap_bronze
    .dropDuplicates(["vehicle_id"])
    .filter("price > 0 and manufacturer IS NOT NULL and model IS NOT NULL")
    .withColumn("year_of_manufacture", sap_bronze["year_of_manufacture"].cast("int"))
    .withColumn("engine_size", sap_bronze["engine_size"].cast("double"))
)
sap_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_sap")

In [0]:
from pyspark.sql import functions as F

# Load Bronze Table
sap_bronze = spark.table("enterprise_modernization.bronze.bronze_sap_1")

# Clean, cast, and enrich for Silver layer
sap_silver = (
    sap_bronze
    .dropDuplicates(["Car_ID"])
    # Filter invalid or null records
    .filter(
        (F.col("Invoice_Amount") > 0) &
        (F.col("Dealer_ID").isNotNull()) &
        (F.col("Region").isNotNull())
    )
    # Cast columns to correct data types
    .withColumn("Car_ID", F.col("Car_ID").cast("int"))
    .withColumn("Dealer_ID", F.col("Dealer_ID").cast("int"))
    .withColumn("Invoice_Amount", F.col("Invoice_Amount").cast("double"))
    .withColumn("Discount_Percent", F.col("Discount_Percent").cast("double"))
    .withColumn("GST_Amount", F.col("GST_Amount").cast("double"))
    .withColumn("Net_Sale", F.col("Net_Sale").cast("double"))
    
    # Standardize string columns (trim + upper)
    .withColumn("Payment_Mode", F.upper(F.trim(F.col("Payment_Mode"))))
    .withColumn("Region", F.upper(F.trim(F.col("Region"))))
    
    # Optional: add derived metrics
    .withColumn("Effective_Amount", F.round(F.col("Net_Sale") - F.col("GST_Amount"), 2))
    .withColumn("Discount_Value", 
        F.round(F.col("Invoice_Amount") * (F.col("Discount_Percent") / 100), 2))
    
    # Optional: add timestamp for auditing
    .withColumn("Updated_Timestamp", F.current_timestamp())
)

# Write the cleaned Silver table
sap_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_sap_1")

print("✅ SAP Bronze → Silver transformation complete and written to silver_sap_1")


### CRM

In [0]:
crm_bronze = spark.table("enterprise_modernization.bronze.bronze_crm")
crm_silver = (
    crm_bronze
    .dropDuplicates(["order_id"])
    .filter("sales > 0 and customer_id IS NOT NULL and vehicle_id IS NOT NULL")
    .withColumn("order_date", crm_bronze["order_date"].cast("date"))
)
crm_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_crm")


In [0]:
crm_bronze = spark.table("enterprise_modernization.bronze.bronze_crm")


In [0]:
from pyspark.sql import functions as F

# Load Bronze CRM data
crm_bronze = spark.table("enterprise_modernization.bronze.bronze_crm_1")

# Transform and clean CRM → Silver
crm_silver = (
    crm_bronze
    # Drop duplicate Car_IDs to keep latest entry
    .dropDuplicates(["Car_ID"])
    
    # Filter invalid or missing rows
    .filter(
        (F.col("Price") > 0) &
        (F.col("Manufacturer").isNotNull()) &
        (F.col("Model").isNotNull())
    )
    
    # Cast types properly
    .withColumn("Car_ID", F.col("Car_ID").cast("int"))
    .withColumn("Engine_size", F.col("Engine_size").cast("double"))
    .withColumn("Year_of_manufacture", F.col("Year_of_manufacture").cast("int"))
    .withColumn("Mileage", F.col("Mileage").cast("int"))
    .withColumn("Price", F.col("Price").cast("double"))
    
    # Clean text columns
    .withColumn("Manufacturer", F.upper(F.trim(F.col("Manufacturer"))))
    .withColumn("Model", F.upper(F.trim(F.col("Model"))))
    .withColumn("Fuel_type", F.upper(F.trim(F.col("Fuel_type"))))
    
    # Add age feature
    .withColumn("Vehicle_Age", F.lit(2024) - F.col("Year_of_manufacture"))
    
    # Add timestamp for lineage
    .withColumn("Updated_Timestamp", F.current_timestamp())
)

# Write Silver layer
crm_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_crm_1")

print("✅ CRM Bronze → Silver transformation complete and written to silver_crm_1")


### Fleet

In [0]:
fleet_bronze = spark.table("enterprise_modernization.bronze.bronze_fleet")
fleet_silver = (
    fleet_bronze
    .dropDuplicates(["device_id", "timestamp"])
    .filter("battery_level >= 0 and fault_detected IN (0, 1)")
    .withColumn("timestamp", fleet_bronze["timestamp"].cast("timestamp"))
)
fleet_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_fleet")


In [0]:
from pyspark.sql import functions as F

# Load Fleet Bronze table
fleet_bronze = spark.table("enterprise_modernization.bronze.bronze_fleet_1")

# Transform Fleet → Silver
fleet_silver = (
    fleet_bronze
    .dropDuplicates(["Car_ID"])
    .filter(
        (F.col("Maintenance_Cost") > 0) &
        (F.col("Fuel_Consumption") > 0)
    )
    
    # Cast data types
    .withColumn("Car_ID", F.col("Car_ID").cast("int"))
    .withColumn("Total_Km", F.col("Total_Km").cast("double"))
    .withColumn("Fuel_Consumption", F.col("Fuel_Consumption").cast("double"))
    .withColumn("Accidents_Count", F.col("Accidents_Count").cast("int"))
    .withColumn("Maintenance_Cost", F.col("Maintenance_Cost").cast("double"))
    
    # Clean text fields
    .withColumn("Insurance_Claim", F.upper(F.trim(F.col("Insurance_Claim"))))
    .withColumn("Fleet_Type", F.upper(F.trim(F.col("Fleet_Type"))))
    
    # Derived columns
    .withColumn("Is_Commercial", F.when(F.col("Fleet_Type") == "COMMERCIAL", F.lit(1)).otherwise(F.lit(0)))
    .withColumn("Has_Insurance_Claim", F.when(F.col("Insurance_Claim") == "YES", F.lit(1)).otherwise(F.lit(0)))
    
    # Data freshness timestamp
    .withColumn("Updated_Timestamp", F.current_timestamp())
)

# Write Silver layer
fleet_silver.write.mode("overwrite").saveAsTable("enterprise_modernization.silver.silver_fleet_1")

print("✅ Fleet Bronze → Silver transformation complete and written to silver_fleet_1")
